In [68]:
import cupy as cp
import time
from gqimax2.mapper import broadcasted_multiplies, broadcasted_multiplies_base4

lambdas = cp.array([1] * 2, dtype=cp.float32)
arrayss = [
    [cp.array([3], dtype=cp.float32), 
     cp.array([1, 2], dtype=cp.float32), 
     ] for i in range(2)
]

# arrayss_list = [
#     [
#         [cp.array([2, 1, 1], dtype=cp.float32),
#          cp.array([1.2], dtype=cp.float32),
#          cp.array([1, 2], dtype=cp.float32),
#          cp.array([3, 4], dtype=cp.float32)]
#         for _ in range(50000)
#     ]
#     for _ in range(10)
# ]
# lambdas_list = [lambdas for _ in range(10)]

if len(arrayss) != lambdas.shape[0]:
    raise ValueError("Kích thước của lambdas và arrayss không khớp.")
start1 = time.time()


result_cp, offsets_result = broadcasted_multiplies(lambdas, arrayss)
print("Thời gian tinhs", time.time() - start1)
# Nếu cần tách kết quả trên GPU
weightss = cp.split(result_cp, offsets_result[1:].tolist())
print(lambdas)
print(arrayss)
print(weightss)
# start1 = time.time()
# for i in range(10):
#     shapes, data, offsets, offsets_result = prepare_data_from_arrayss(arrayss_list[i])
#     result_cp, offsets_result = broadcasted_multiplies_cuda(lambdas_list[i], shapes, data, offsets, offsets_result)
# print("Thời gian tinhs2", time.time() - start1)


# for i, weights in enumerate(weightss):
#     print(f"Kết quả bộ {i}: {weights.shape}")

Thời gian tinhs 0.0019989013671875
[1. 1.]
[[array([3.], dtype=float32), array([1., 2.], dtype=float32)], [array([3.], dtype=float32), array([1., 2.], dtype=float32)]]
[array([3., 6.], dtype=float32), array([3., 6.], dtype=float32)]


In [69]:
lambdas

array([1., 1.], dtype=float32)

In [70]:
import cupy as cp

import time

start1 = time.time()
arrayss = [
    [
        cp.array([2], dtype=cp.int32), 
     	cp.array([0, 1], dtype=cp.int32),
	] for _ in range(2)
]

# n_jobs = 10
# arrayss_list = [
#     [
#         [cp.array([2, 1, 1], dtype=cp.float32),
#          cp.array([1.2], dtype=cp.float32),
#          cp.array([1, 2], dtype=cp.float32),
#          cp.array([3, 4], dtype=cp.float32)]
#         for _ in range(5000)
#     ]
#     for _ in range(n_jobs)
# ]




# print("Khởi tạo mảng:", time.time() - start1)

# start1 = time.time()
# for i in range(10):
# 	result_gpu, offsets_result = encode_base4s_gpu(arrayss_list[i])
# print("encode:", time.time() - start1)

result_gpu, offsets_result = broadcasted_multiplies_base4(arrayss)
positionss = cp.split(result_gpu, offsets_result[1:].tolist())
# for i, res in enumerate(positionss):  # In 3 bộ đầu tiên để kiểm tra
#     print(f"Kết quả bộ {i}: {res}")
positionss

[array([8, 9]), array([8, 9])]

In [71]:
arrayss

[[array([2]), array([0, 1])], [array([2]), array([0, 1])]]

In [72]:
positionss

[array([8, 9]), array([8, 9])]

In [73]:
weightss

[array([3., 6.], dtype=float32), array([3., 6.], dtype=float32)]

In [74]:
import cupy as cp
from gqimax2.mapper import sum_distributions



positions_gpu, totals_gpu = sum_distributions(weightss, positionss)

print("Positions:", positions_gpu)
print("Total values:", totals_gpu)

Positions: [8 9]
Total values: [ 6. 12.]


In [86]:
import cupy as cp
import numpy as np
import itertools

# Example input
arrayss = [
    [
        cp.array([4], dtype=cp.int32),
        cp.array([0], dtype=cp.int32),
        cp.array([0], dtype=cp.int32)
    ],
    [
        cp.array([2], dtype=cp.int32),
        cp.array([0, 1], dtype=cp.int32),
        cp.array([0, 1], dtype=cp.int32)
    ]
]

import cupy as cp
import numpy as np
import itertools

arrayss = [
    [
        cp.array([1,2], dtype=cp.int32),
        cp.array([3], dtype=cp.int32),
        cp.array([4,5], dtype=cp.int32)
    ],
    [
        cp.array([1], dtype=cp.int32),
        cp.array([2, 3], dtype=cp.int32),
        cp.array([4, 5], dtype=cp.int32)
    ]
]

import cupy as cp

def broadcast_batches(arrayss):
    results = []
    for arrays in arrayss:
        # Expand all arrays to shape (1, ..., N, ..., 1)
        reshaped = [a.reshape((1,) * i + (-1,) + (1,) * (len(arrays) - i - 1)) for i, a in enumerate(arrays)]
        # Broadcast to common shape
        broadcasted = cp.broadcast_arrays(*reshaped)
        # Stack along last axis to get combinations
        stacked = cp.stack(broadcasted, axis=-1)
        # Reshape to (num_combinations, num_features)
        results.append(stacked.reshape(-1, len(arrays)))
    return results

print(broadcast_batches(arrayss))

[array([[1, 3, 4],
       [1, 3, 5],
       [2, 3, 4],
       [2, 3, 5]]), array([[1, 2, 4],
       [1, 2, 5],
       [1, 3, 4],
       [1, 3, 5]])]


In [100]:
import cupy as cp
import cupyx as cpx
# Step 1: Simulate your ragged_matrix and ragged_indices in GPU
ragged_matrix = [
    cp.array([1, 2, 3, 4], dtype=cp.int32),
    cp.array([1, 2], dtype=cp.int32)
]

ragged_indices = [
    cp.array([[0,1], [0,2], [1,1], [1,3]], dtype=cp.int32),
    cp.array([[0,1], [1,2]], dtype=cp.int32)
]


def sum_distributions(weightss, ragged_indices):
    # Step 1: Flatten inputs
    values = cp.concatenate(weightss)
    idx_concat = cp.concatenate(ragged_indices)

    # Special case: only one value
    if values.size == 1:
        return idx_concat.reshape(1, -1), values

    # General case
    base = 10000
    keys = idx_concat[:, 0] * base + idx_concat[:, 1]
    unique_keys, inverse = cp.unique(keys, return_inverse=True)

    sums = cp.zeros(len(unique_keys), dtype=cp.int32)
    cpx.scatter_add(sums, inverse, values)

    decoded_indices = cp.stack([unique_keys // base, unique_keys % base], axis=1)
    return decoded_indices, sums

decoded_indices, sums = sum_distributions(ragged_matrix, ragged_indices)
print("Indices:\n", decoded_indices)
print("Values:\n", sums)


Indices:
 [[0 1]
 [0 2]
 [1 1]
 [1 2]
 [1 3]]
Values:
 [2 2 3 2 4]


In [ ]:
def sum_distributions(weightss, ragged_indices):

    return decoded_indices, sums


ragged_matrix = [
    cp.array([-0.36763045,  0.9023816 , -0.22484507]),
]

ragged_indices = [
    cp.array([[1, 0, 0], [2,0,0], [3, 0, 0]], dtype=cp.int32)
]

       
decoded_indices, sums = sum_distributions(ragged_matrix, ragged_indices)
print("Indices:\n", decoded_indices)
print("Values:\n", sums)

Indices:
 [[1 0 0]
 [2 0 0]
 [3 0 0]]
Values:
 [-0.36763045  0.9023816  -0.22484507]


In [102]:
import cupy as cp

def sum_by_indices(ragged_indices, ragged_matrix):
    """
    Sum values from ragged_matrix based on corresponding indices in ragged_indices.
    If indices are identical, their values are summed; otherwise, they are appended to the result.

    Args:
        ragged_indices (list of cp.ndarray): List of CuPy arrays containing multi-dimensional indices.
        ragged_matrix (list of cp.ndarray): List of CuPy arrays containing values to sum.

    Returns:
        tuple: (unique_indices, summed_values)
            - unique_indices: CuPy array of unique indices.
            - summed_values: CuPy array of summed values corresponding to unique indices.
    """
    # Concatenate all indices into a single array of shape (N, k)
    all_indices = cp.vstack(ragged_indices)
    
    # Concatenate all values into a single array of shape (N,)
    all_values = cp.concatenate(ragged_matrix)
    
    # Total number of index-value pairs
    N = all_indices.shape[0]
    
    # Sort indices lexicographically by rows
    sort_idx = cp.lexsort(all_indices.T[::-1])
    sorted_indices = all_indices[sort_idx]
    sorted_values = all_values[sort_idx]
    
    # Identify where rows differ to mark the start of each unique group
    diff = (sorted_indices[1:] != sorted_indices[:-1]).any(axis=1)
    is_new = cp.concatenate([cp.array([True]), diff])
    split_idx = cp.where(is_new)[0]
    
    # Extract unique indices
    unique_indices = sorted_indices[split_idx]
    
    # Compute sums for each group using cumulative sum
    cumsum = cp.cumsum(sorted_values)
    total_cumsum = cp.concatenate([cp.array([0]), cumsum])
    group_sums = total_cumsum[cp.concatenate([split_idx[1:], cp.array([N])])] - total_cumsum[split_idx]
    
    return unique_indices, group_sums

# Example usage:
if __name__ == "__main__":
    # Example 1
    ragged_matrix1 = [
        cp.array([1, 2, 3, 4]),
        cp.array([1, 2])
    ]
    ragged_indices1 = [
        cp.array([[0,1], [0,2], [1,1], [1,3]], dtype=cp.int8),
        cp.array([[0,1], [1,2]], dtype=cp.int8)
    ]
    indices1, values1 = sum_by_indices(ragged_indices1, ragged_matrix1)
    print("Example 1 Indices:\n", indices1)
    print("Example 1 Values:\n", values1)
    
    # Example 2
    ragged_matrix2 = [
        cp.array([-0.36763045, 0.9023816, -0.22484507])
    ]
    ragged_indices2 = [
        cp.array([[1,0,0], [2,0,0], [3,0,0]], dtype=cp.int8)
    ]
    indices2, values2 = sum_by_indices(ragged_indices2, ragged_matrix2)
    print("Example 2 Indices:\n", indices2)
    print("Example 2 Values:\n", values2)

Example 1 Indices:
 [[0 1]
 [0 2]
 [1 1]
 [1 2]
 [1 3]]
Example 1 Values:
 [2 2 3 2 4]
Example 2 Indices:
 [[1 0 0]
 [2 0 0]
 [3 0 0]]
Example 2 Values:
 [-0.36763045  0.9023816  -0.22484507]
